# Homework 2

## Environment Setup

In [1]:
%%shell
gdown --quiet 1ay5DcH64Qao1HR7CQnR6Cl1hbBMgGqXj
gdown --quiet 13BozEl3JtS43Xuu2Ek9IwMULpWjPH4VC
gdown --quiet 1It6GP8O2JqkmUtZKbYp1kpwpuwOXlLps
pip --quiet install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 7.4 MB/s eta 0:00:00


In [2]:
SAT_FN = 'SAT_Results.csv'
HSD_FN = 'DOE_High_School_Directory_2014-2015.csv'
ENRON_FN = 'enron_mails_small.csv'

import pyspark
from pyspark.sql import SparkSession
sc = pyspark.SparkContext.getOrCreate()
spark = SparkSession.builder.getOrCreate()
spark

## Task 1 (5 points)

Your task is to do Task 2 of Lab 5 (see **`BDM_Lab05_Spark.pdf`**). Below is a step-by-step walkthrough of the solution using RDD (using the approach detailed in the hint file **`BDM_Lab05_Spark_Keys.pdf`). Please complete each sub-task that is marked with `[TO DO]` (step `D`, `E`, and `G`). For those cells without `[TO DO]`, you are expected to run it without any edits and without any error, with the provided output.

### A. Load **SAT Scores** Dataset without Header

In [3]:
# We read the SAT score to our RDD. Note that the use_unicode can be
# changed accordingly to your data file to handle Unicode. If you cannot
# parse your data due to an 'utf8' or 'ascii' decoding issue, it might
# be a good thing to try flipping the use_unicode parameter here.

sat = sc.textFile(SAT_FN, use_unicode=True).cache()
satHeader = sat.first().split(',')
A = sat.filter(lambda x: not x.startswith('DBN,SCHOOL'))

# This line for us to list the column index and column names to see
# which column we need to use for our task. In this case, we're
# interested in the number of test takers (#2) and the math score (#4).
display(list(enumerate(satHeader)))
A.take(3)

[(0, 'DBN'),
 (1, 'SCHOOL NAME'),
 (2, 'Num of SAT Test Takers'),
 (3, 'SAT Critical Reading Avg. Score'),
 (4, 'SAT Math Avg. Score'),
 (5, 'SAT Writing Avg. Score')]

['02M047,47 THE AMERICAN SIGN LANGUAGE AND ENGLISH SECONDARY SCHOOL,16,395,400,387',
 '21K410,ABRAHAM LINCOLN HIGH SCHOOL,475,396,437,393',
 '21K412/21K411,ABRAHAM LINCOLN YABC/LEARNING TO WORK GED AT ABRAHAM LINCOLN,s,s,s,s']

### B. Extract `DBN`, `Num of SAT Test Takers` and `SAT Math Avg. Score` columns

Here is a different approach to read the SAT score file using `map()` instead of `mapPartitionsWithIndex()`. Please note the use of `csv.reader` with a single input line.

In [4]:
import csv

B = A.map(lambda x: next(csv.reader([x]))) \
    .filter(lambda x: x[2]!='s') \
    .map(lambda x: (x[0], (int(x[2]), int(x[4]))))

B.take(3)

[('02M047', (16, 400)), ('21K410', (475, 437)), ('30Q301', (98, 440))]

### C. Load **High School Directory** Dataset and check its header

In [5]:
# Here we do the same thing with the school directory data
C = sc.textFile(HSD_FN, use_unicode=True).cache()
hsdHeader = C.first().split(',')
list(enumerate(hsdHeader))

[(0, 'dbn'),
 (1, 'school_name'),
 (2, 'boro'),
 (3, 'building_code'),
 (4, 'phone_number'),
 (5, 'fax_number'),
 (6, 'grade_span_min'),
 (7, 'grade_span_max'),
 (8, 'expgrade_span_min'),
 (9, 'expgrade_span_max'),
 (10, 'bus'),
 (11, 'subway'),
 (12, 'primary_address_line_1'),
 (13, 'city'),
 (14, 'state_code'),
 (15, 'zip'),
 (16, 'website'),
 (17, 'total_students'),
 (18, 'campus_name'),
 (19, 'school_type'),
 (20, 'overview_paragraph'),
 (21, 'program_highlights'),
 (22, 'language_classes'),
 (23, 'advancedplacement_courses'),
 (24, 'online_ap_courses'),
 (25, 'online_language_courses'),
 (26, 'extracurricular_activities'),
 (27, 'psal_sports_boys'),
 (28, 'psal_sports_girls'),
 (29, 'psal_sports_coed'),
 (30, 'school_sports'),
 (31, 'partner_cbo'),
 (32, 'partner_hospital'),
 (33, 'partner_highered'),
 (34, 'partner_cultural'),
 (35, 'partner_nonprofit'),
 (36, 'partner_corporate'),
 (37, 'partner_financial'),
 (38, 'partner_other'),
 (39, 'addtl_info1'),
 (40, 'addtl_info2'),
 (4

### [TO DO] D. Extract `DBN`, `bus` and `subway` columns from the HSD data

In [9]:
# Complete the extractFeatures function to get the desire output
def extractFeatures(partId, rows):
    if partId==0:
        next(rows) # avoiding header here
    reader = csv.reader(rows)
    for fields in reader:
        if len(fields) < 12: # Solving index problem
            continue
        dbn, bus, subway = fields[0], fields[10], fields[11]
        yield (dbn, bus, subway)

### DO NOT EDIT BELOW
D = C.mapPartitionsWithIndex(extractFeatures) 
D.take(5)

[('01M292',
  'B39, M14A, M14D, M15, M15-SBS, M21, M22, M9',
  'B, D to Grand St ; F to East Broadway ; J, M, Z to Delancey St-Essex St'),
 ('01M448',
  'M14A, M14D, M15, M21, M22, M9',
  'F to East Broadway ; J, M, Z to Delancey St-Essex St'),
 ('01M450',
  'M101, M102, M103, M14A, M14D, M15, M15-SBS, M2, M23, M3, M8, M9',
  '6 to Astor Place ; L to 1st Ave'),
 ('01M509',
  'B39, M103, M14A, M14D, M15, M15-SBS, M21, M22, M8, M9',
  'B, D to Grand St ; F, J, M, Z to Delancey St-Essex St'),
 ('01M539',
  'B39, M14A, M14D, M21, M22, M8, M9',
  'F, J, M, Z to Delancey St-Essex St')]

### [TO DO] E. Convert the bus and subway description into the list of bus and subway lines

For the subway, we do not care about the direction of the lines, e.g. `B, D to Grand St` should be treated as just `B, D`.

In [10]:
def ConvertBusSubway(i):
    dbn, bus, subway = i
    bus_lines = list(map(lambda x: x.strip(), bus.split(','))) #Splitting by comma
    subway_lines = []
    for line in subway.split(';'):
        subway_lines.extend(list(map(lambda x: x.strip(), line.split('to')[0].split(',')))) #Splitting by comma & to
    return (dbn, (bus_lines, subway_lines))

# Derive E from above
E = D.map(lambda i: ConvertBusSubway(i))

### DO NOT EDIT BELOW
E.take(5)

[('01M292',
  (['B39', 'M14A', 'M14D', 'M15', 'M15-SBS', 'M21', 'M22', 'M9'],
   ['B', 'D', 'F', 'J', 'M', 'Z'])),
 ('01M448',
  (['M14A', 'M14D', 'M15', 'M21', 'M22', 'M9'], ['F', 'J', 'M', 'Z'])),
 ('01M450',
  (['M101',
    'M102',
    'M103',
    'M14A',
    'M14D',
    'M15',
    'M15-SBS',
    'M2',
    'M23',
    'M3',
    'M8',
    'M9'],
   ['6', 'L'])),
 ('01M509',
  (['B39', 'M103', 'M14A', 'M14D', 'M15', 'M15-SBS', 'M21', 'M22', 'M8', 'M9'],
   ['B', 'D', 'F', 'J', 'M', 'Z'])),
 ('01M539',
  (['B39', 'M14A', 'M14D', 'M21', 'M22', 'M8', 'M9'], ['F', 'J', 'M', 'Z']))]

### F. Join `B` (SAT) and `E`(HSD) datasets so that the score is together with the list of bus lines and subway lines.

We no longer need to the DBN information after this.

In [11]:
F = B.join(E).values()
F.take(5)

[((59, 374),
  (['B12', 'B16', 'B35', 'B41', 'B44', 'B49'], ['2', '5', 'B', 'Q'])),
 ((135, 492),
  (['B62',
    'Q100',
    'Q101',
    'Q102',
    'Q103',
    'Q32',
    'Q39',
    'Q60',
    'Q66',
    'Q67',
    'Q69'],
   ['7', 'N', 'Q', 'E', 'M', 'R', 'F', 'G'])),
 ((228, 456),
  (['Bx15',
    'Bx19',
    'Bx33',
    'M10',
    'M100',
    'M101',
    'M104',
    'M11',
    'M2',
    'M3',
    'M4',
    'M5'],
   ['1', 'A', 'D', 'B', 'C'])),
 ((270, 441),
  (['M101', 'M102', 'M15', 'M2', 'M31', 'M50', 'M57', 'Q101', 'Q32', 'Q60'],
   ['4', '5', '6', 'N', 'Q', 'R', '6', 'E', 'M', 'F'])),
 ((60, 391), (['Bx12', 'Bx12-SBS', 'Bx26', 'Bx39', 'Bx8'], ['2', '5']))]

### [TO DO] G. Group the scores by the subway line and bus line so that we can compute the average by line
Note that, one school may belong to multiple lines, so we need to somehow replicate the school score for each bus line. The output of this step should be in the key/value pair format with the key is the line, and the value is the average score.

To make it easier, please store the bus and subway data in `G_bus` and `G_subway` respectively.

In [12]:
# Creatting a function to count average score.
def cal_avg(data):
    return data.map(lambda x: (x[1], (x[0][1] * x[0][0], x[0][0]))) \
               .reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1])) \
               .mapValues(lambda x: x[0]/x[1])

# Create G_bus and G_subway here as RDDs
G_bus = cal_avg(F.flatMapValues(lambda x: x[0]))
G_subway = cal_avg(F.flatMapValues(lambda x: x[1]))

## DO NOT EDIT BELOW
display(G_bus.take(5))
display(G_subway.take(5))

[('B44', 465.0336676217765),
 ('Q66', 461.55149181905676),
 ('Q69', 444.03056234718827),
 ('M100', 427.83587443946186),
 ('M3', 432.531451213472)]

[('N', 493.5055292259084),
 ('4', 495.29238227146817),
 ('L', 426.3222871994802),
 ('J', 439.1299656694458),
 ('B', 491.95760524225574)]

### H. Get the line with the highest score

In [13]:
display(G_bus.max(lambda x: x[1]))
display(G_subway.max(lambda x: x[1]))

('S1115', 612.2545811518324)

('3', 513.4009556313994)

# Task 2 (5 points)
You are asked to implement the Social Triangle example discussed in the MapReduce lecture but using Spark. Given the email dataset, please list all "reciprocal" relationships in the company. Recall that:

If A emails B and B emails A, then A and B is *reciprocal*.

If A emails B but B doesn’t email A, then A and B is *directed*.

**Dataset:** We will use a subset of the open [Enron Email Dataset](https://www.cs.cmu.edu/~./enron/ "Enron Email Dataset"), which contains approximately 10,000 simplified email headers from the Enron Corporation. A subset of the data is available as **enron_mails_small.csv**

The file contains 3 columns *Date*, *From*, and *To*. Their description is as follows:

|Column name|Description|
|--|--|
|Date |The date and time of the email, in the format YYYY-MM-DD hh-mm-ss, <br />e.g. "1998-10-30 07:43:00" |
|From |The sender email address, <br />e.g. "mark.taylor@enron.com" |
|To | A list of recipients' email addresses separated by semicolons ';', <br />e.g. "jennifer.fraser@enron.com;jeffrey.hodge@enron.com" |

Note that, we only care about users employed by Enron, i.e. only relationships where email addresses end with *'@enron.com'*.

The expected output is also provided below. For each reciprocal relationship, please output a tuple consisting of two strings (or a two column dataframe), e.g. **('Jane Doe', 'John Doe')**. The names should be presented in the lexical order, i.e. there will not be a `('John Doe','Jane Doe')` since `'Jane'` is ordered before `'John'`.

Though the dataset only contains email addresses, not actual names, we're assuming that the email aliases were created based on their name. For example:

|Email Address|Converted Name|
|--|--|
|mark.taylor@enron.com|Mark Taylor|
|alan.aronowitz@enron.com|Alan Aronowitz|
|marc.r.cutler@enron.com|Marc R Cutler|
|hugh@enron.com|Hugh|

### You can choose to use either Spark's RDD or Spark's DataFrame in this task.
Regardless of your choice, you must perform all computation using Sparks' transformations, i.e. data must be read directly from the input file into your RDD and DataFrame and stay there for the entire computation.

### Using RDD

In [14]:
mail_csv = 'enron_mails_small.csv'
mail = sc.textFile(mail_csv, use_unicode=True).cache()
list(enumerate(mail.first().split(',')))

[(0, 'Date'), (1, 'From'), (2, 'To')]

In [15]:
def extractEmails(partId, rows):
    if partId == 0:
        next(rows)
    reader = csv.reader(rows)
    for fields in reader:
        if len(fields) == 2:
            continue
        From, To = fields[1], fields[2]
        if not (From.endswith('@enron.com') and To.endswith('@enron.com')):
            continue
        if any(char.isdigit() for char in From):
            continue
        # Splitting "To" email addresses using ";" separator and filter for Enron emails
        To_list = [to_email.strip() for to_email in To.split(';') if to_email.strip().endswith('@enron.com') and not any(char.isdigit() for char in to_email.strip())]
        for to_email in To_list:
            yield (From, to_email)

A2 = mail.mapPartitionsWithIndex(extractEmails)
A2.take(5)

[('mark.taylor@enron.com', 'shari.stack@enron.com'),
 ('mark.taylor@enron.com', 'yao.apasu@enron.com'),
 ('mark.taylor@enron.com', 'paul.simons@enron.com'),
 ('mark.taylor@enron.com', 'justin.boyd@enron.com'),
 ('mark.taylor@enron.com', 'tana.jones@enron.com')]

In [17]:
def extract_name(email):
    return tuple(map(lambda x: x.split('@')[0].replace('.', ' ').title(), email))
B2= A2.map(lambda x: extract_name(x))
B2.take(5)

[('Mark Taylor', 'Shari Stack'),
 ('Mark Taylor', 'Yao Apasu'),
 ('Mark Taylor', 'Paul Simons'),
 ('Mark Taylor', 'Justin Boyd'),
 ('Mark Taylor', 'Tana Jones')]

In [18]:
# Remove duplicates
C2 = B2.distinct()
C2.count()

3390

In [19]:
# Sort by the From column
D2 = C2.sortByKey()
D2.take(5)

[('Andrew Lewis', 'Derek Davies'),
 ('Angela Mcculloch', 'Ione Irvine'),
 ('Angela Mcculloch', 'Attila Pazmandi'),
 ('Angela Mcculloch', 'Dan Dietrich'),
 ('Angela Mcculloch', 'Tara Sweitzer')]

In [20]:
def interactions(names):
    sorted_names = tuple(sorted(names))
    return (sorted_names, 1)

# Countting the occurrences of each email pair
E2 = D2.map(lambda x: interactions(x)) \
       .reduceByKey(lambda x, y: x + y)
E2.take(5)

[(('Andrew Lewis', 'Derek Davies'), 1),
 (('Angela Mcculloch', 'Ione Irvine'), 1),
 (('Angela Mcculloch', 'Attila Pazmandi'), 1),
 (('Angela Mcculloch', 'Dan Dietrich'), 1),
 (('Angela Mcculloch', 'Tara Sweitzer'), 1)]

In [21]:
# Your code to read and process data into rddTask2
rddTask2 = E2.filter(lambda x: x[1] >= 2) \
       .map(lambda x: x[0]) \
       .sortByKey()

# DO NOT EDIT BELOW
rddTask2.collect()

[('Brenda Whitehead', 'Elizabeth Sager'),
 ('Carol Clair', 'Sara Shackleton'),
 ('Carol Clair', 'Debra Perlingiere'),
 ('Carol Clair', 'Tana Jones'),
 ('Carol Clair', 'Mark Taylor'),
 ('Carol Clair', 'Richard Sanders'),
 ('Debra Perlingiere', 'Kevin Ruscitti'),
 ('Drew Fossum', 'Susan Scott'),
 ('Elizabeth Sager', 'Janette Elbertson'),
 ('Elizabeth Sager', 'Mark Haedicke'),
 ('Elizabeth Sager', 'Mark Taylor'),
 ('Elizabeth Sager', 'Richard Sanders'),
 ('Eric Bass', 'Susan Scott'),
 ('Fletcher Sturm', 'Greg Whalley'),
 ('Fletcher Sturm', 'Sally Beck'),
 ('Gerald Nemec', 'Susan Scott'),
 ('Grant Masson', 'Vince Kaminski'),
 ('Greg Whalley', 'Richard Sanders'),
 ('Janette Elbertson', 'Mark Taylor'),
 ('Janette Elbertson', 'Richard Sanders'),
 ('Liz Taylor', 'Mark Haedicke'),
 ('Mark Haedicke', 'Mark Taylor'),
 ('Mark Haedicke', 'Richard Sanders'),
 ('Mark Haedicke', 'Twanda Sweet'),
 ('Mark Haedicke', 'Michelle Cash'),
 ('Mark Taylor', 'Tana Jones'),
 ('Mark Taylor', 'Sara Shackleton'),
 

## Using DataFrame

In [ ]:
# Your code to read and process data into dfTask2
dfTask2 = ...

# DO NOT EDIT BELOW
dfTask2.show(n=50)

+--------------------+-----------------+
|            Person 1|         Person 2|
+--------------------+-----------------+
|    Brenda Whitehead|  Elizabeth Sager|
|         Carol Clair|Debra Perlingiere|
|         Carol Clair|      Mark Taylor|
|         Carol Clair|  Richard Sanders|
|         Carol Clair|  Sara Shackleton|
|         Carol Clair|       Tana Jones|
|   Debra Perlingiere|   Kevin Ruscitti|
|         Drew Fossum|      Susan Scott|
|     Elizabeth Sager|Janette Elbertson|
|     Elizabeth Sager|    Mark Haedicke|
|     Elizabeth Sager|      Mark Taylor|
|     Elizabeth Sager|  Richard Sanders|
|           Eric Bass|      Susan Scott|
|      Fletcher Sturm|     Greg Whalley|
|      Fletcher Sturm|       Sally Beck|
|        Gerald Nemec|      Susan Scott|
|        Grant Masson|   Vince Kaminski|
|        Greg Whalley|  Richard Sanders|
|   Janette Elbertson|      Mark Taylor|
|   Janette Elbertson|  Richard Sanders|
|          Liz Taylor|    Mark Haedicke|
|       Mark Hae